In [ ]:
# Gym stuff
import gym
import gym_anytrading

# Stable baselines - rl stuff
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO

# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from datetime import date

# Download stock data to pandas dataframe
from jugaad_data.nse import stock_df

In [ ]:
df = stock_df(symbol="SBIN", from_date=date(2022,1,1),
            to_date=date(2023,1,1), series="EQ")

In [ ]:
df.sort_values('DATE', ascending = True, inplace = True)

df.set_index('DATE', inplace=True)

df = df.drop(['SERIES', 'PREV. CLOSE', 'LTP', 'VWAP', '52W H', '52W L', 'VALUE', 'NO OF TRADES','SYMBOL'], axis=1)

df.rename(columns = {'OPEN' : 'Open', 'HIGH' : 'High', 'LOW' : 'Low','CLOSE' : 'Close', 'VOLUME' : 'Volume'},  inplace = True)

df.head(5)

In [ ]:
env_maker = lambda: gym.make('stocks-v0', df=df, frame_bound=(5,200), window_size=5)
env = DummyVecEnv([env_maker])

model = PPO('MlpPolicy', env, verbose=1) 
model.learn(total_timesteps=1000000)

In [ ]:
env = gym.make('stocks-v0', df=df, frame_bound=(201,248), window_size=5)
obs = env.reset()
while True: 
    obs = obs[np.newaxis, ...]
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        print("info", info)
        break
        
plt.figure(figsize=(25,25))
plt.cla()
env.render_all()
plt.show()